In [2]:
import streamlit as st
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import OllamaEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.llms import Ollama

e:\Projects\Bhaiya_project_pacs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document

def load_valid_pdfs(directory):
    documents = []
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        if file_name.lower().endswith('.pdf'):
            try:
                loader = PyPDFLoader(file_path)
                docs = loader.load()
                documents.extend(docs)
            except Exception as e:
                print(f"Error loading {file_name}: {e}")
    return documents

# Load valid PDFs
pdf_directory = "./downloaded_pdfs/usefuldocs"
documents = load_valid_pdfs(pdf_directory)

if documents:
    print(f"Successfully loaded {len(documents)} documents.")
else:
    print("No valid PDFs were loaded.")


Successfully loaded 2100 documents.


In [3]:
if documents:
    print(f"Successfully loaded {len(documents)} documents.")
else:
    print("No valid PDFs were loaded.")

Successfully loaded 2100 documents.


In [4]:
documents

[Document(metadata={'source': './downloaded_pdfs/usefuldocs\\000120364.pdf', 'page': 0}, page_content=' \n－1－ \n（Provisional Translation） \n \nBasic Policy concerning the Promotion of Contracts considering Reduction of \nEmissions of Greenhouse Gases and Others by the State and Other Entities \n \n1. The basic direction for the pr omotion of contracts that consider reductions of \nemissions of greenhouse gases and others. \n(1) Background and significance of the promotion of green contracts  \nIn light of  the scale and severity of its anticipated impacts, global warming is the \nmost significant environmental issue threaten ing the foundations of human survival.  \nAccording to the "Summary for Policymakers of the Working Group I Contribution to the Sixth \nAssessment Report of the Intergovernmental Panel on Climate Change (IPCC)" released in August \n2021, it is unequivocal that human influence has warmed the atmosphere, ocean and land. \nWidespread and rapid changes in the atmospher

In [5]:
# Define Chroma database directory
chroma_db_dir = "./chroma_db"

In [4]:
from dotenv import load_dotenv
load_dotenv()
# Load the Google API key from .env
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Directory for Chroma persistence
PERSISTENT_DIR = "chroma_db"
os.makedirs(PERSISTENT_DIR, exist_ok=True)  # Create the directory if it doesn't exist

In [5]:
# Initialize the Chroma vector store with persistent directory

chroma_db = Chroma(
    collection_name="my_documents",
    persist_directory=PERSISTENT_DIR,
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY )
)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2596\3483200515.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(


In [19]:
# Add chunks to Chroma
chroma_db.add_documents(documents)
st.success("Embeddings created successfully!")

2024-11-18 00:54:52.283 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-18 00:54:52.450 
  command:

    streamlit run e:\Projects\Bhaiya_project_pacs\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-11-18 00:54:52.450 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [6]:
# Retrieve relevant documents from Chroma
retriever = chroma_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import LLMChain
from langchain.chains import create_retrieval_chain
# Create the prompt template
template = ChatPromptTemplate.from_template("""
You are an expert climate change policies assistant who help peoples to answer their queries. Given the following retrieved documents:
{context}

Based on this context, answer the following question:
{input}
always follow these instructions:
1. Answers peoples greetings politely
2. Answer people queries like you have created this answer without adding this kind of lines in the answer (Based on the provided text)                       
3.If the retrieved documents do not contain sufficient information to answer the question, say: "Your data doesn't contain this information but according to my knowledge" Then, provide an answer based on your own knowledge, ensuring to clarify that your response is informed by your understanding beyond the retrieved content.
""")

chain = create_stuff_documents_chain(llm, template)
qa_chain = create_retrieval_chain(retriever, chain)



In [9]:

query = "who is john cena"
retrieved_docs = retriever.get_relevant_documents(query)
retrieved_text = "\n".join([doc.page_content for doc in retrieved_docs])
try:
    response = qa_chain.invoke({"input": query, "context": retrieved_text})
except:
    print("We are working on the issues")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2596\2542953682.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [10]:
response['answer']

"Your data doesn't contain this information but according to my knowledge, John Cena is a professional wrestler, actor, and rapper.  He is best known for his career in WWE (World Wrestling Entertainment).\n"

In [2]:
import sqlite3
print(f"SQLite version: {sqlite3.sqlite_version}")


SQLite version: 3.45.1
